In [1]:
import torch
 
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
 
print("using", device, "device")

using cuda device


In [2]:
!pip install -U pillow
!pip install rembg

In [3]:
import requests
from PIL import Image
from io import BytesIO
import os
from rembg.bg import remove

In [4]:
def process_image(input_path, output_folder, background_img):
    # Load the image
    img = Image.open(input_path)
    
    # Save the original image
    original_filename = os.path.basename(input_path)
    img.save(os.path.join(output_folder, 'original', original_filename), format='JPEG')
    
    # Remove the background
    with open(input_path, 'rb') as f:
        input_data = f.read()
        subject = remove(input_data, alpha_matting=True)
    
    # Save the masked image
    output_path = os.path.join(output_folder, 'masked', original_filename)
    with open(output_path, 'wb') as f:
        f.write(subject)
    
    # Open masked image
    foreground_img = Image.open(output_path)
    
    # Paste masked image onto background
    background_img.paste(foreground_img, (0, 0), foreground_img)
    
    # Save final image
    background_img.save(os.path.join(output_folder, 'processed', original_filename), format='JPEG')


In [5]:
def process_folder(input_folder, output_folder, background_folder):
    os.makedirs(os.path.join(output_folder, 'original'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'masked'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'processed'), exist_ok=True)
    
    background_images = os.listdir(background_folder)
    
    # Iterate over each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpeg") or filename.endswith(".jpg"):
            input_path = os.path.join(input_folder, filename)
            
            # Loop through each background image
            for bg_image in background_images:
                background_path = os.path.join(background_folder, bg_image)
                
                # Process the image with the current background
                process_image(input_path, output_folder, Image.open(background_path))

In [6]:
input_folder = r"E:\My projects\Rembg\frames_extraction_HD\fg_frames"
output_folder = r"D:\My projects\Rembg\output_frames_rembg"
background_folder = r"E:\My projects\Rembg\frames_extraction_HD\bg_frames"

process_folder(input_folder, output_folder, background_folder)

KeyboardInterrupt: 